<a href="https://colab.research.google.com/github/BhaskarFysik/AlgoShared/blob/main/Share_copy_of_bh_fyers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fyers-apiv2

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install pandas_ta

In [ ]:
pip install backtesting

In [ ]:
app_id=''
app_secret=''
redirect_uri=''

#libraries
from fyers_api import fyersModel,accessToken
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt


#login
def get_access_token():
    if not os.path.exists("access_token.txt"):
        session=accessToken.SessionModel(client_id=app_id,
        secret_key=app_secret,redirect_uri=redirect_uri,
        response_type='code', grant_type='authorization_code')
        response = session.generate_authcode()
        print("login url=",response)
        auth_code =input("Enter auth_code")

        session.set_token(auth_code)
        access_token = session.generate_token()["access_token"]
        with open("access_token.txt","w") as f:
            f.write(access_token)
    else:
        with open("access_token.txt","r") as f:
            access_token=f.read()
    return access_token

fyers = fyersModel.FyersModel(client_id=app_id, token=get_access_token(),log_path="")
#login complete


#print(fyers.get_profile())
#print(fyers.funds())
print(fyers.holdings()['holdings'])



[{'holdingType': 'HLD', 'quantity': 1, 'costPrice': 67.1, 'marketVal': '67.90', 'remainingQuantity': 1, 'pl': 0.8, 'ltp': 67.9, 'id': 1, 'fyToken': '10100000001624', 'exchange': 10, 'isin': 'INE242A01010', 'symbol': 'NSE:IOC-EQ'}]


In [ ]:
#date time coversion function
def date_time(x):
    import datetime
    return(datetime.datetime.fromtimestamp(int(x)).strftime('%d-%m-%Y  %H:%M:%S'))


#historical data
data = {"symbol":"NSE:ITC-EQ","resolution":"1","date_format":"1","range_from":"2022-09-20","range_to":"2022-10-17","cont_flag":"1"}
#print(fyers.history(data))
history_data=fyers.history(data)
#print(history_data.keys())
candles=history_data['candles']
#print(candles)
df=pd.DataFrame(candles)
df.columns=('Timestamp','Open','High','Low','Close','Volume')
#df['Timestamp']=itc['Timestamp'].apply(date_time)
print(df)


       Timestamp    Open    High     Low   Close  Volume
0     1663645500  338.75  338.75  336.90  337.40  325866
1     1663645560  337.30  337.50  336.35  336.70  102097
2     1663645620  336.70  336.85  336.30  336.70   87102
3     1663645680  336.70  337.20  336.70  336.80  107102
4     1663645740  336.80  337.30  336.60  337.10  145288
...          ...     ...     ...     ...     ...     ...
7120  1666000500  332.50  332.50  332.00  332.05   58142
7121  1666000560  332.00  332.05  331.90  332.00   50706
7122  1666000620  331.90  332.00  331.90  332.00   89500
7123  1666000680  332.00  332.50  331.95  331.95   76111
7124  1666000740  332.00  332.70  332.00  332.40   15177

[7125 rows x 6 columns]


In [ ]:
#backtesting

from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest

def RSI(values,n):
     return ta.rsi(close=df["Close"], length=n)
def SMA(values,n):
     return pd.Series(values).rolling(n).mean()



class RsiOscillator(Strategy):
    upper_bound=55
    lower_bound=45
    n1=15
    n2=20
    def init(self):
        self.rsi=self.I(RSI,self.data.Close,14)
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self):

        if crossover(self.rsi,self.upper_bound) and self.sma2>self.sma1:
            if self.position.is_long:
              self.position.close()
              self.sell()
        elif crossover(self.lower_bound,self.rsi) and self.sma2<self.sma1:
            if self.position.is_short or not self.position:
              self.position.close()
              self.buy()

bt=Backtest(df,RsiOscillator, cash=10000)
"""stats=bt.optimize(
    upper_bound=range(10,85,5),
    lower_bound=range(10,85,5),
    n1=range(5, 30, 5),
    n2=range(10, 70, 5),
    maximize='Equity Final [$]',
    constraint=lambda param: param.upper_bound>param.lower_bound and param.n1 < param.n2)"""
stats=bt.run()
print(stats)
bt.plot()

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                    7124.0
Duration                               7124.0
Exposure Time [%]                   99.368421
Equity Final [$]                      11594.7
Equity Peak [$]                       11678.0
Return [%]                             15.947
Buy & Hold Return [%]               -1.481921
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.393226
Avg. Drawdown [%]                   -0.262233
Max. Drawdown Duration                 1127.0
Avg. Drawdown Duration               50.23913
# Trades                                157.0
Win Rate [%]                        68.152866
Best Trade [%]                       1.811648
Worst Trade [%]                     -1.894056
Avg. Trade [%]                    

Row(id='1480', ...)

In [ ]:
print(stats["_strategy"].upper_bound)
print(stats["_strategy"].lower_bound)
print(stats["_strategy"].n1)
print(stats["_strategy"].n2)